<a href="https://colab.research.google.com/github/K-S-Samarth-Rao/InternshipStudio/blob/master/Intern_Samarth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **CONTEXT** **:** The bank has a growing customer base. The bank wants to increase borrowers (asset customers) base to bring in more  loan business and earn more through the interest on loans. So , the bank wants to convert the liability based customers to personal loan customers. (while retaining them as depositors). A campaign that the bank ran last year for liability customers showed a healthy conversion rate of over 9% success. The department wants you to build a model that will help them identify the potential customers who have a higher probability of purchasing the loan. This will increase the success ratio while at the same time reduce the cost of the campaign.


**Objective :** The classification goal is to predict the likelihood of a liability customer buying personal
loans.




##                   **1.1 Importing the Required Libraries and Datasets** 





In [ ]:
import io                                                                            # For Stream Handling
import numpy as np                                                                   # For Linear Algebra
import pandas as pd                                                                  # For Data Processing
import seaborn as sns                                                                # For Informative Plots
import matplotlib.pyplot as plt                                                      # For Plots
%matplotlib inline
sns.set(color_codes=True)
from scipy.stats import zscore
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from google.colab import files                                                       # Uploading the file containing the Data.
uploaded = files.upload()               
data = pd.read_excel('/content/Bank_Personal_Loan_Modelling.xlsx','Data')        # Reading/Accessing the Data

### **1.2 Preview of First 5 Rows**

In [ ]:
data.head(5)

### **1.3 Preview of Last 5 Rows**

In [ ]:
data.tail(5)

### **1.4 Shape of the Dataset**

In [ ]:
print(data.shape)                                         # Size of the Dataset i.e. (Rows,Columns)
print(data.shape[1])                                      # Number of Features in the Dataset
print(data.shape[0])                                      # Number of Samples in the Dataset

### **1.5 Various Features of the Dataset**

In [ ]:
data.columns

### **1.6 Details on the Features of the Dataset**
1) ID: Customer ID.

2) Age: Customer's age in completed years.

3) Experience: Years of professional experience.

4) Income: Annual income of the customer.

5) ZIP Code: Home Address.

6) Family: Family size of the customer

7) CCAvg: Avg. spending on credit cards per month ($) 

8) Education : Education Level--> 1:Undergraduate 2:Graduate                    3:Advanced/          Professional

9) Mortgage: Value of house mortgage if any. ($)

10) Personal Loan: Did this customer accept the personal loan offered in the last campaign?

11) Securities Account: Does the customer have a securities account with the bank?

12) CD Account: Does the customer have a certificate of deposit (CD) account with
the bank?

13) Online: Does the customer use internet banking facilities?

14) Credit card: Does the customer use a credit card issued by the bank?

### **1.7 Checking the DataTypes of Fields and non null count**

In [ ]:
data.info()                   # We observe that there are no null values in any of the feature. Moreover all Fields are Numerical in Datatype

### **1.8 Statistical Information**

In [ ]:
data.describe().T  # We observe that the minimum experience is negative which is impossible. The rest looks fine

Here , 

1) Ordinal Categorical Variables are : Family , Education .

2) The Nominal Variables are : ID , ZIP Code .

3) Internal Variables are : Age , Experience , Income , CCAvg , Mortage .

### **2.1 Cleaning the Data**

In [ ]:
data[data['Experience']<0]['Experience'].count()  # We find there are 52 entires (approx. 1% ) in 'Experience' having negative values. These need to be fixed.

### **2.2 Finding any relation(if exists) among the Fields**

In [ ]:
sns.pairplot(data.iloc[:,1:])

From the plots its clear that the field Age and Experience form a good pair . They seem to have a strong linear relationship between them.(This relation can help us in refilling the faulty negative Experience values).

Other interesting relationships to be noted are between Income & CCAvg and Income & Mortgage. 

The Scatterplots clearly tell us that CCAvg and Mortgage values are lesser than the income.

### **3.1 Finding Unique Data** 

In [ ]:
data.apply(lambda x: len(x.unique()))

### **3.2 Inference from the plots of '2.2'**
**Age** - This feature is normally distributed with the majority of customers falling between 30 years and 60 years of age. We can confirm this by looking at the describe statement above , which tells that the **Mean** and the **Median** are almost equal.

**Experience** - This feature is normally distributed with more customers having experience starting from 8 years. Here the **Mean** and the **Median** are equal. There are negative values in Experience, which is a faulty input.

**Income** - This feature is skewed positively. Majority of the customers have income between 45K and 55K as the **Mean** is greater than the **Median**.

**CCAvg** - This feature is also skewed positively. Here the average spending is between 0K to 10K and majority of them spend less than 2.5K .

**Mortage** - In this feature , 70% of the customers have a mortage of less than 40K .However ,the max value is 635K .

In [ ]:
# Applying Categorical Feature. Here , Features having less than 25 values are considered as categorical
cat_features = [feature for feature in data.columns if data[feature].nunique()<25]
print('Categorical Features are : ' , cat_features)
data[cat_features].head()


In [ ]:
# Applying Continuous Feature
cont_features = [feature for feature in data.columns if feature not in cat_features + ['ID']]
cont_features

### **3.3 Understanding the distribution of each Categorical Feature**

In [ ]:
for feature in cat_features:
    sns.countplot(data[feature])
    plt.show()

**1)** We observe here that the number of customers who took loan last year(our target variable) were very less(<500) compared to the set of customers who haven't taken the loan(>4500). The Dataset is highly imbalanced.

**2)** Very few customers have Securities account or CD account (<500).

**3)** Majority of the customers do not own Credit Cards as well (<3500).

### **3.4 Understanding the distribution of each Continuous Feature**

In [ ]:
for feature in cont_features :
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(12,5))
    sns.distplot(data[feature], kde=False,ax=ax1)
    sns.boxplot(data[feature], ax=ax2)
    plt.show()

The Observations are :-

**1)** Income and CCAvg are positively skewed distribution with some outliers towards the higher end.Hence probability for it being incorrect is less as there are very few people with outstanding salaries.

**2)** Mortage is also having some outliers towards the higher end. It can be seen from describe function that 75% of customers had Mortage value less than 101 indicating the high peak on the distribution plot, even though maximum was 635. Similar to above , probability for it being incorrect is less.

**3)** Majority of people are aged between 35 to 55 with experience between 10 to 30 .They have an evident relationship.

### **3.5 Variation of CCAvg with Income**

In [ ]:
sns.scatterplot(x='Income' , y='CCAvg',hue='Personal Loan' ,data=data)

In [ ]:
plt.figure(figsize=(15,8))
sns.pointplot(x='Income',y='CCAvg', data=data, estimator=np.median,ci=None)

The Observations are :-

**1)** People having more income tend to spend more on their credit card.

**2)** From the 1st figure it is clear that people with higher income opted for Personal Loan.

**3)** People dont tend to spend more on credit card than their income (scatterplot triangular).

### **3.6 Variation of Income with Education and Experience**

In [ ]:
sns.boxplot(x='Education',y='Income', hue='Personal Loan',data=data)

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x='Experience',y='Income',hue='Personal Loan',data=data)

We observe an interesting thing that people with lower education are earning more income. And Income doesn't vary based on Experience. Also,Personal Loan is independent of Education and Experience.

Personal Loan is dependent on the income though.It is evident from the higher band boxplots for Personal Loan that people with higher income tend to opt for Personal Loan.

From the Zip Code boxplot, there is an outlier. 

### **3.7 Checking for unique Zip Code entries**

In [ ]:
print(data['ZIP Code'].min())
print(data['ZIP Code'].max())
data['ZIP Code'].unique()

One Zipcode has just 4 digits which is a faulty input. We shall change it to 5 digits by appending a '0' to maintain the geographic area of atleast first 4 digits.

### **3.8 Influence of Securities Account ,Family Size,CD Account on Personal Loan**

In [ ]:
sns.countplot(x="Securities Account",data=data,hue="Personal Loan")

We observe that Majority of customers who dont have loan have Securities account.

In [ ]:
sns.countplot(x='Family', data=data,hue='Personal Loan',palette='Set1')

We observe that family size doesn't have any impact on Personal Loan.But we find that families with size of 3 are more likely to take Loan.


In [ ]:
sns.countplot(x='CD Account',data=data,hue='Personal Loan')

We observe that majority of Customers who does not possess CD Account , do not possess any Loan as well. And, almost all customers having CD Account have Loan.

### **3.9 Heatmap for further analysis**

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(data.corr(),annot = True)


From the Heatmap, we observe:-

1) Age and Experience are highly corelated(0.99).

2) CCAvg and Income are correlated.

3) Income and Personal Loan are correlated.

### **3.10** Checking relation between Mortgage and Credit-Card

In [ ]:
print('Number of customers with No House Mortgage :',data[data['Mortgage']==0].shape[0],' out of', data.shape[0])
print('Number of customers with No spending on credit card :',data[data['CCAvg']==0].shape[0], 'out of' , data.shape[0])

We observe some mistake here. Because, we saw earlier that the number of customers who did not use credit card was higher(>3500).But the CCAvg says only 106 customers did not have spendings on credit card.

CCAvg is right (from Heatmap) , hence reset the values of credit card.

### **4.1 Applying necessary transformation to required features**

In [ ]:
# If CCAvg > 0 then, set the Credit Card as 1 else 0
data['CreditCard']=np.where(data['CCAvg']>0 ,1,0)

### **4.2 Correcting the faulty ZIP Code entry**

In [ ]:
data['ZIP Code']= data['ZIP Code'].apply(lambda zip:zip*10 if zip<10000 else zip)

In [ ]:
sns.boxplot(data['ZIP Code'])                # Analysing the Corrected ZIP Code

### **4.3 Correcting the Negative Experience Values**

In [ ]:
print('Number of rows with Negative Experience : ' , sum(data['Experience']<0))
print('Ages of rows with Negative Experience : ',data[data['Experience']<0].Age.unique())

In [ ]:
sns.lmplot('Age','Experience',data=data[data['Experience']>0])

Since , there exists a strong Linear relationship between Experience and Age.Let's use a Linear model to predict the values of Negative Experience using Age.

In [ ]:
lr_model = LinearRegression()
lr_model.fit(data[['Age']],data['Experience'])
data.loc[data['Experience']<0,'Experience']=lr_model.predict(data[data['Experience']<0].loc[:,'Age':'Age'])

In [ ]:
print('Number of rows with Negative Experience : ' , sum(data['Experience']<0))
print('Ages of rows with Negative Experience : ',data[data['Experience']<0].Age.unique())

The faulty negative values are corrected by predictions from the model. We will not be able to compute Experience for people with Age <= 25 using the model as it will predict negative values.

Let us replace the leftover negative Experience values by 0 to maintain the relationship.

In [ ]:
data.loc[data['Experience']<0,'Experience']=0
data[data['Experience']<0]

In [ ]:
sns.scatterplot(data['Age'],data['Experience'])

### **4.4 Dropping the Noisy Features**

In [ ]:
X = data.drop(['ID','Age','ZIP Code','Personal Loan','CreditCard'], axis= 1)
y=data['Personal Loan']
X

The above Features have high Correlation (VIF values).

### **5.1  Splitting the data into Training and Testing set in the ratio of 70:30  respectively.**

In [ ]:
train_set, test_set= train_test_split(data.drop(['ID','Experience'],axis=1),test_size=0.3,random_state=100)

ID doesnt help in giving any info about customers buying Personal Loan.

Experience and Age are highly related . Hence Age alone is enough.

In [ ]:
train_labels = train_set.pop('Personal Loan')
test_labels =test_set.pop('Personal Loan')

### **Decision Tree Classifier**

In [ ]:
DATA_model=DecisionTreeClassifier(criterion='entropy',max_depth=3)
DATA_model.fit(train_set, train_labels)

In [ ]:
DATA_model.score(test_set,test_labels)

In [ ]:
y_predict=DATA_model.predict(test_set)
y_predict[:5]

In [ ]:
test_set.head(5)

### **Naive Bayes**

In [ ]:
naive_model=GaussianNB()
naive_model.fit(train_set , train_labels)

prediction = naive_model.predict(test_set)
naive_model.score(test_set,test_labels)

### **Random Forest Classifier**

In [ ]:
randomforest_model= RandomForestClassifier(max_depth=2,random_state=0)
randomforest_model.fit(train_set,train_labels)

In [ ]:
Importance= pd.DataFrame({'Importance': randomforest_model.feature_importances_*100}, index=train_set.columns)
Importance.sort_values('Importance',axis=0,ascending=True).plot(kind='barh',color='b',)

In [ ]:
predicted_random=randomforest_model.predict(test_set)
randomforest_model.score(test_set,test_labels)

### **KNN (K - Nearest Neighbour)**

In [ ]:
train_set_indep= data.drop(['Experience','ID'],axis=1).drop(labels="Personal Loan",axis=1)
train_set_dep=data["Personal Loan"]
X=np.array(train_set_indep)
Y=np.array(train_set_dep)
X_Train= X[ :3500, :]
X_Test =X[3501: , :]
Y_Train = Y[:3500,]
Y_Test = Y[3501: ,]

In [ ]:
knn =KNeighborsClassifier(n_neighbors=21, weights= 'uniform', metric='euclidean')
knn.fit(X_Train,Y_Train)
predicted = knn.predict(X_Test)
from sklearn.metrics import accuracy_score
accu = accuracy_score(Y_Test,predicted)
print(accu)

### **MODEL Comparision**

In [ ]:
from sklearn import model_selection
models  = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART',DecisionTreeClassifier()))
models.append(('NB',GaussianNB()))
models.append(('RF', RandomForestClassifier()))
# Evaluating each model in turn
results= []	
names = []
scoring = 'accuracy'
for name, model in models:
        kfold = model_selection.KFold(n_splits=10, random_state =12345)
        cv_results = model_selection.cross_val_score(model,X,y,cv =kfold , scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg="%s: %f (%f)" % (name,cv_results.mean() , cv_results.std())
        print(msg)
# Box Plot Algorithm Comparision   
fig = plt.figure()
fig.suptitle('Algorithm Comparision')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()     

**Conclusion -** It seems that **Decision Tree** algorithm has the highest accuracy .

But before choosing it as our final model, let's check some more models.

In [ ]:
X = data.drop(['ID','Age','ZIP Code','Personal Loan','CreditCard'], axis= 1)
y=data['Personal Loan']
X

In [ ]:
# Split the Data into Test and Train Dataset

X_train, X_test,y_train,y_test = train_test_split(X,y , test_size= 0.3, random_state=1234)

print('Number of 0s in Train Dataset : ', y_train.value_counts()[0])
print('Numberof 1s in Train dataset : ', y_train.value_counts()[1])
print('--------------')
print('Number of 0s in Test Dataset : ', y_test.value_counts()[0])
print('Number of 1s in Test Dataset : ', y_test.value_counts()[1])

X_train= X_train.apply(zscore)
X_test= X_test.apply(zscore)

In [ ]:
# Function to print the different metrics such as Confusion matrix , roc, Accuracy, Precision , Recall etc

def printModelMetrics(model , X_train , X_test , y_train , y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    print('Train set accuracy = ' , metrics.accuracy_score(y_train,y_train_pred))
    print('Test set accuracy = ' , metrics.accuracy_score(y_test , y_test_pred))
    print(metrics.classification_report(y_test, y_test_pred))

    cm = metrics.confusion_matrix(y_test , y_test_pred)
    cm = pd.DataFrame(cm,columns=['Predicted No Loan','Predicted Loan'], index=['Truth No Loan', 'Truth Loan'])
    sns.heatmap(cm, annot = True ,fmt ='g', cbar=False)
    plt.show()

    y_test_proba = model.predict_proba(X_test)
    y_test_proba = y_test_proba[:,1]
# Generate a no Skill Prediction (Majority Class)
    ns_probs = [0 for _ in range(len(y_test))]
  # Calculate Scores
    ns_auc = metrics.roc_auc_score(y_test, ns_probs)
    lr_auc = metrics.roc_auc_score(y_test , y_test_proba)
  # Summarize Scores
    print('ROC AUC = %.3f' % (lr_auc))
  # Calculate roc Curves
    ns_fpr, ns_tpr, _ = metrics.roc_curve(y_test,ns_probs)
    lr_fpr, lr_tpr, _ = metrics.roc_curve(y_test, y_test_proba)
  # Plot the roc Curve for the Model
    plt.plot(ns_fpr,ns_tpr,linestyle='--',label='No Skill')
    plt.plot(lr_fpr,lr_tpr,marker='.',label='Model Skill')
  # Axis Labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
  # Show the legend
    plt.legend()
  # Show the Plot
    plt.show()

### **Logistic Regression**

In [ ]:
lr_model = LogisticRegression(solver='liblinear')
lr_model.fit(X_train, y_train)
printModelMetrics(lr_model, X_train, X_test, y_train, y_test)

Test Accuracy is 94% .

We observe that the recall of customers who opted the Loan(target) is very less(54%) , i.e. we predicted 46% of the people to be Non Loan takers when they actually took Loan. This affetcs the marketing campaign and should be reduced.

### **Naive Bayes**

In [ ]:
nb_model = BernoulliNB()

nb_model.fit(X_train ,y_train)
printModelMetrics(lr_model, X_train,X_test,y_train,y_test)

### **KNN (K - Nearest Neighbour)**

In [ ]:
X_trainScaled =X_train.apply(zscore)
X_testScaled = X_test.apply(zscore)
model = KNeighborsClassifier(n_neighbors=20,weights='distance')
model.fit(X_trainScaled,y_train)
printModelMetrics(model,X_trainScaled,X_testScaled,y_train,y_test)

### **SVM**

In [ ]:
model = svm.SVC(gamma = 0.025 , C=3 , probability=True)
model.fit(X_train,y_train)
printModelMetrics(model,X_train,X_test,y_train,y_test)

The highest recall we could get was 68% with an accuracy of 96% from SVM Model.

Let's try to improve our models. Initially we observed that the data was highly imbalanced with majority people as Non Loan customers. Let's balance this data by oversampling the loan classes and then rebuild the models.

In [ ]:
# Oversampling the positive classes in the Data

# Separating Majority and Minority Classes
df_majority = data[data['Personal Loan']==0]
df_minority = data[data['Personal Loan']==1]

# Unsample Minority Classes
df_minority_upsampled = resample(df_minority, replace =True , n_samples = df_majority.shape[0], random_state =123)
df_upsampled = pd.concat([df_majority,df_minority_upsampled])
df_upsampled['Personal Loan'].value_counts()
X= df_upsampled.drop(['ID','ZIP Code','Personal Loan','Experience'],axis=1)
y= df_upsampled['Personal Loan']
X_train, X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=1)

print('Number of 0s in Train Dataset : ' , y_train.value_counts()[0])
print('Number of 1s in Train Dataset  : ' , y_train.value_counts()[1])
print('--------------')
print('Number of 0s in Test Dataset : ', y_test.value_counts()[0])
print('Number of 1s in Test Dtataset :', y_test.value_counts()[1])

### **Logistic Regression**

In [ ]:
lr_model = LogisticRegression(solver = 'liblinear')
lr_model.fit(X_train, y_train)
printModelMetrics(lr_model, X_train , X_test, y_train , y_test)

### **Naive Bayes**

In [ ]:
nb_model = BernoulliNB()
nb_model.fit(X_train, y_train)
printModelMetrics(lr_model , X_train , X_test , y_train, y_test)

### **KNN (K - Nearest Neighbour)**

In [ ]:
X_trainScaled =X_train.apply(zscore)
X_testScaled = X_test.apply(zscore)
model = KNeighborsClassifier(n_neighbors=20,weights='distance')
model.fit(X_trainScaled,y_train)
printModelMetrics(model,X_trainScaled,X_testScaled,y_train,y_test)

### **SVM**

In [ ]:
model = svm.SVC(gamma=0.2,C=5,probability=True)
model.fit(X_train,y_train)
printModelMetrics(model,X_train,X_test,y_train,y_test)

### **Conclusion -** The aim of the universal bank is to convert there liability customers into Loan customers. They want to set up  new marketing campaign; hence they need information about the connection between the variables given in the Data. From th various algorithms used ,  With the recall of 100% and Accuracy of 99.5% , **SVM** is definitely the best model to chooose for our campaign .